In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pandas as pd
import skfuzzy as fuzz
import os
from skfuzzy import control as ctrl
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
import os

In [2]:
train_df = pd.read_csv('Thesis/train.csv')

In [4]:
amount = ctrl.Antecedent(np.arange(0, 10000, 1000), 'amount')
cross_border = ctrl.Antecedent(np.arange(0, 2, 1), 'cross_border')
country_risk = ctrl.Antecedent(np.arange(0, 2, 1), 'country_risk')  # For country risk like Iran, Syria, North Korea
pep_involvement = ctrl.Antecedent(np.arange(0, 2, 1), 'pep_involvement')
transaction_type = ctrl.Antecedent(np.arange(0, 3, 1), 'transaction_type')  # 0: domestic, 1: international, 2: digital

risk = ctrl.Consequent(np.arange(0, 101, 1), 'risk')

# Membership Functions
amount.automf(3, names=['low', 'medium', 'high'])
cross_border['domestic'] = fuzz.trimf(cross_border.universe, [0, 0, 0.5])
cross_border['international'] = fuzz.trimf(cross_border.universe, [0.5, 1, 1])

country_risk['low'] = fuzz.trimf(country_risk.universe, [0, 0, 0.5])
country_risk['high'w] = fuzz.trimf(country_risk.universe, [0.5, 1, 1])

pep_involvement['non_pep'] = fuzz.trimf(pep_involvement.universe, [0, 0, 0.5])
pep_involvement['pep'] = fuzz.trimf(pep_involvement.universe, [0.5, 1, 1])

transaction_type['domestic'] = fuzz.trimf(transaction_type.universe, [0, 0, 0.5])
transaction_type['international'] = fuzz.trimf(transaction_type.universe, [0.5, 1, 1.5])
transaction_type['digital'] = fuzz.trimf(transaction_type.universe, [1.5, 2, 2])

risk['low'] = fuzz.trimf(risk.universe, [0, 0, 50])
risk['medium'] = fuzz.trimf(risk.universe, [20, 50, 80])
risk['high'] = fuzz.trimf(risk.universe, [60, 100, 100])

# Fuzzy rules
rule1 = ctrl.Rule(amount['high'] | cross_border['international'], risk['high'])
rule2 = ctrl.Rule(amount['medium'] & cross_border['domestic'], risk['medium'])
rule3 = ctrl.Rule(amount['low'] & (cross_border['domestic'] | cross_border['international']), risk['low'])
rule4 = ctrl.Rule(country_risk['high'], risk['high'])
rule5 = ctrl.Rule(pep_involvement['pep'], risk['high'])
rule6 = ctrl.Rule(transaction_type['digital'], risk['medium'])
rule7 = ctrl.Rule(transaction_type['international'], risk['high'])

# Control system setup
aml_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
aml_sim = ctrl.ControlSystemSimulation(aml_control)

# Function to apply fuzzy system to each transaction
def evaluate_transaction(row):
    reasons = []

    # Determine the transaction type
    if row['Transaction_Type'] in ['digital', 'CRYPTO-TRANSFER', 'PAYMENT']:
        transaction_type_value = 2
    elif row['Transaction_Type'] == 'international':
        transaction_type_value = 1
    else:
        transaction_type_value = 0

    # Determine PEP involvement
    pep_involvement_value = 1 if row['Bene_Is_Pep'] or row['Sender_Is_Pep'] else 0

    aml_sim.inputs({
        'amount': row['USD_Amount'],
        'cross_border': 1 if row['Sender_Country'] != row['Bene_Country'] else 0,
        'country_risk': 1 if row['Bene_Country'] in ['Iran', 'Syria', 'North-Korea'] else 0,
        'pep_involvement': pep_involvement_value,
        'transaction_type': transaction_type_value
    })

    try:
        # Compute risk score
        aml_sim.compute()
        risk_score = aml_sim.output['risk']
        if risk_score >= 60:  # Assuming 60 as the threshold for high risk
            if row['USD_Amount'] >= amount.universe.max():
                reasons.append("High Amount")
            if 1 if row['Sender_Country'] != row['Bene_Country'] else 0 == 1:
                reasons.append("Cross Border Transaction")
            if 1 if row['Bene_Country'] in ['Iran', 'Syria', 'North-Korea'] else 0 == 1:
                reasons.append("High Risk Country")
            if pep_involvement_value == 1:
                reasons.append("PEP Involvement")
            if transaction_type_value == 2:
                reasons.append("Digital Transaction")
        return risk_score, reasons
    except Exception as e:
        print(f"Error during risk evaluation: {e}")
        return np.nan, reasons  # Return NaN if an error occurs (no rules fired)

train_df['risk_score'], train_df['fuzzy_result'] = zip(*train_df.apply(evaluate_transaction, axis=1))
train_df['fuzzy_result'] = train_df['fuzzy_result'].apply(
    lambda x: 'None' if isinstance(x, list) and not x else x if isinstance(x, list) else str(x)
)

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [5]:
train_df.to_csv('Thesis/train_with_fuzzy_results.csv', index=False)